In [3]:
import galois 
import numpy as np
#from py_ecc.bn128 import curve_order
import sympy as sp
#smart optimization for fast curve-point multiplication
#GF = galois.GF(curve_order, primitive_element=5, verify=False)
GF = galois.GF(79)
curve_order = 79

In [4]:

# EXERCISE 1
def mul_point(point, scalar):
    return point[0], point[1] * scalar

def lagrange_interpolate(points):
    x = sp.Symbol('x')
    lagrange_poly = sp.interpolate(points, x)
    return lagrange_poly

def phi(vector, to_equation : bool = False):
    # Assuming phi is the Lagrange interpolation
    eq = lagrange_interpolate(vector)
    if to_equation:
        x = sp.Symbol('x')
        #return as a python function
        return sp.lambdify(x, eq)
    else:
        return eq

# Test vectors and scalar

vector = np.random.randint(0, 100, size=10)
vector = [(coord_a, coord_b) for coord_a, coord_b in zip(vector[:len(vector)//2], vector[len(vector)//2:])]
c = 3

# Compute phi(c * vector)
c_vector = [mul_point(x, c) for x in vector]
phi_c_vector = phi(c_vector)

# Compute c * phi(vector)
phi_vector = phi(vector)
c_phi_vector = c * phi_vector

print("phi(c * vector):")
print(phi_c_vector)
print("\nc * phi(vector):")
print(c_phi_vector)
print("\nEquality check:")
print(phi_c_vector == c_phi_vector)




phi(c * vector):
516487*x**4/878830560 - 26774183*x**3/292943520 + 229771493*x**2/54926910 - 4444806473*x/73235880 + 375375391/1220598

c * phi(vector):
516487*x**4/878830560 - 26774183*x**3/292943520 + 229771493*x**2/54926910 - 4444806473*x/73235880 + 375375391/1220598

Equality check:
True


In [10]:
def lagrange_interpolate(points):
    x = sp.Symbol('x')
    lagrange_poly = sp.interpolate(points, x)
    return lagrange_poly.as_poly().all_coeffs()

# Test the updated function
x = [1, 2, 3, 4]
y = [1, 4, 9, 16]

#lagrange coeffs
coeffs = lagrange_interpolate(list(zip(x, y)))
print(coeffs)

[1, 0, 0]


In [ ]:
# EXERCISE 2
def interpolate_column(col, upper_bound : int) -> np.array:
    xs = np.arange(1, upper_bound + 1)
    lagrange_poly = lagrange_interpolate(list(zip(xs, col)))
    return lagrange_poly
    
    #return galois.lagrange_poly(xs, col)
import numpy as np
import random
import functools
import typing as tp

def matrix_to_polynomials(matrix : np.array) -> np.array:
    func = functools.partial(interpolate_column, upper_bound=matrix.shape[1])
    return np.apply_along_axis(func, 0, matrix)
    
# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 100
y = 100

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"

def qap(
    A : np.array, 
    B : np.array, 
    C : np.array, 
    w : np.array, 
    point : tuple[int, int]
):

    A_poly = tp.cast(np.array, matrix_to_polynomials(A))
    B_poly = tp.cast(np.array, matrix_to_polynomials(B))
    C_poly = tp.cast(np.array, matrix_to_polynomials(C))

    n_rows = A.shape[0]
    term1 = C_poly.dot(w)
    term2 = A_poly.dot(w)
    term3 = B_poly.dot(w)
    
    x = sp.Symbol('x')
    t = sp.prod(x - i for i in range(1, n_rows + 1))
    
    h = (term1 * term2 - term3) // t
    
    left_side = term1 * term2
    right_side = term3 + h * t
    
    left_side_fn = sp.lambdify(x, left_side)
    right_side_fn = sp.lambdify(x, right_side)
    
    assert np.isclose(left_side_fn(point[0]), right_side_fn(point[0]), rtol=1e-9), "Left side is not approximately equal to right side"
    return True

print(qap(A, B, C, w, (5, 100)))

In [ ]:
#exercise 3
from functools import partial
def finite_field_interpolate_column(col, upper_bound : int):
    xs = GF(np.arange(1, upper_bound + 1))
    return galois.lagrange_poly(xs, col)

def mat_to_finite_field(mat : np.array) -> np.array:
    return (mat + curve_order) % curve_order



def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def qap_finite_field(A, B, C, w, point):
    n_rows = A.shape[0]
    
    def interpolate_column(col):
        xs = GF(np.arange(1, n_rows + 1))
        return galois.lagrange_poly(xs, col)

    A_polys = np.apply_along_axis(interpolate_column, 0, A)
    B_polys = np.apply_along_axis(interpolate_column, 0, B)
    C_polys = np.apply_along_axis(interpolate_column, 0, C)

    term1 = inner_product_polynomials_with_witness(A_polys, w)
    term2 = inner_product_polynomials_with_witness(B_polys, w)
    term3 = inner_product_polynomials_with_witness(C_polys, w)

    t = galois.Poly([1], field=GF)
    for i in range(1, n_rows + 1):
        t *= galois.Poly([1, curve_order - i], field=GF)

    h = (term1 * term2 - term3) // t

    left_side = term1 * term2
    right_side = term3 + h * t

    print("Left side:", left_side)
    print("Right side:", right_side)

    assert left_side == right_side, "Left side is not equal to right side"
    return True

# Call the function
F_A = GF(mat_to_finite_field(A))
F_B = GF(mat_to_finite_field(B))
F_C = GF(mat_to_finite_field(C))
F_W = GF(mat_to_finite_field(w))


qap_finite_field(F_A, F_B, F_C, F_W, (5, 100))